### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2023 ###


# Visually-Consistent Image Morphing for Kernel Improvement   #

#### Cole Dilanni  (diianni@wisc.edu)
#### Ilay Raz (email address)
#### Nitzan Orr   (nitzan@cs.wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
  1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

Our project is designing an image distance metric which takes two images and finds how to warp one to best align with the other using mixed integer programming. We achieve our results by assigning each pixel binary variables which determine where in the warped image they will map.

Image similarity algorithms are important for many tasks such as image recognition, image retrieval, and object tracking. The most simple image distance is the pixel-wise $L_2$ distance in which two image matricies of the same size are subtracted from each other and the resulting $L_2$ is their dissimilarity. This example highlights that an image similarity algorithm should return a higher value for pairs of images considered dissiimilar and a lower value for images which are very similar/the same.

One problem with a basic $L_2$ distance metric is the "rigidness" of the pixels. This is to say that two images are only considered similar if their values are similar and their pixel positioning matches exactly. A failure case would be a checkerboard image compared to the same image shifted right one pixel. Even though the content remains the same (there has only been a small translation), the $L_2$ distance metric would return an extremely high value since the pixel positioning does not perfectly align.

To this end, we propose a more general $L_2$ distance metric for computer vision which allows the pixels of one image to shift in a way which best aligns with the comparison image. These shifts must follow certain rules when shifting an image A to best align with image B:
- All pixels from A must map to a point in the shifted version of A
- All pixels in B must have a corresponding value in the shifted version of A
- Pixels cannot cross during shift ($Apixel_left$ cannot be further right than $Apixel_right$ in the final shifted version of the image)

To test our algorithm, we will use the MNIST dataset (dataset for handwritten digits 0-9). This dataset is good because it's low resolution (for fast testing), and handwritten digits have the same semantic information (0-9), but are often warped compared to one another, given handwriting differences.







Over the past decade in the Machine Learning field, Neural Networks have become increasingly good at a variety of tasks, including image classification and object detection. One of the parts that is “learned” by the network is the kernel, or a small visual “template” or “recipe” that lets neural networks detect certain patterns in an image. Kernels are matched against incoming images and output how closely the image matches the kernel itself. For example, a kernel that takes on the template of a circle would output higher scores when it is matched against images of wheels or the number zero. While good at pattern-matching, Kernels also function quite rigidly. A kernel looking for circular patterns would find an off-angle shot of a wheel or a messy hand-written digit “0” to be completely foreign and unknown. As such, we have devised a method to enable flexibility of kernels to morph into nearby shapes.

Our method, which we call Spatially Transformed and Enhanced Pixel-wise High-dimensional ENcoding (STEPHEN), is an optimization-based image-morphing method (thanks ChatGPT). We take a source image and target image, and compute the transformation of each pixel between the two images while maintaining the visual geometry consistent. The output of the algorithm is similar to that of optical flow but rather than retroactively tracking pixels, it transforms them in the tracked direction. 

#####  The first few sentences should give a quick overview of the entire project. Then, elaborate with a description of the problem that will be solved, a brief history (with [citations](https://en.wikipedia.org/wiki/Citation)) of how the problem came about, why it's important/interesting, and any other interesting facts you'd like to talk about. You should address and explain where the problem data is coming from (research? the internet? synthetically generated?) Also give an outline of the rest of the report.

##### This section should be 300-600 words long, and **should be accessible to a general audience** (don't assume your reader has taken the class!). Feel free to include images if you think it'll be helpful:

![fixit flowchart][flow]

For more help on using Markdown, see [this reference](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).

[flow]: https://s-media-cache-ak0.pinimg.com/736x/f5/75/c5/f575c53b93724808c6f0211890a54900.jpg

## 2. Mathematical model ##

A discussion of the modeling assumptions made in the problem (e.g. is it from physics? economics? something else?). Explain the decision variables, the constraints, and the objective function. Finally, show the optimization problem written in standard form. Discuss the model type (LP, QP, MIP, etc.). Equations should be formatted in $\LaTeX$ within the IJulia notebook. For this section you may **assume the reader is familiar with the material covered in class**.

Here is an example of an equation:

$$
\begin{bmatrix}
  1 & 2 \\
  3 & 4
\end{bmatrix}
\begin{bmatrix} x \\ y \end{bmatrix} =
\begin{bmatrix} 5 \\ 6 \end{bmatrix}
$$

And here is an example of an optimization problem in standard form:

$$
\begin{aligned}
\underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& f_0(x) \\
\text{subject to:}\qquad& f_i(x) \le 0 && i=1,\dots,m\\
& h_j(x) = 0 && j=1,\dots,r
\end{aligned}
$$

For some quick tips on using $\LaTeX$, see [this cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).

## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

In [1]:
# this is a code block
using JuMP, Clp
m=Model(Clp.Optimizer)

things = [:horses, :donkeys, :goats]  # these are the things 
@variable(m, x[things] >= 0)          # the quantities of each of the things (can't be negative)
@constraint(m, sum(x) <= 10)          # we can't have any more than 10 things total
@objective(m, Max, x[:horses])        # we want to maximize the number of horses
optimize!(m)

for i in things
    println("The total number of ", i, " is: ", value(x[i]))     # print result
end

The total number of horses is: 10.0
The total number of donkeys is: 0.0
The total number of goats is: 0.0
Coin0506I Presolve 0 (-1) rows, 0 (-3) columns and 0 (-3) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 10
Coin0511I After Postsolve, objective 10, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 10 - 0 iterations time 0.002, Presolve 0.00


Remember to make sure your code compiles! I will be running your code!

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

We have proposed STEPHEN, an optimization-based pixel transformation method that maintains visual semantic information. Our method successfully showed how optimization can be used to “massage” similar looking images which can be used in a number of ways. First, we have shown a proof of concept that this method can be used to improve digit classification. Second, STEPHEN can be used to make kernels more flexible by allowing a single kernel to take on different forms, while remaining semantically consistent. 

In Neural Network training, data augmentation is a widely used and respected (by some) technique that our method could aid with. In the future our method could be used to transform input images to introduce higher variation in the training data and allow networks to better generalize. Another possible future direction is… 


##### Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.